## Classical SDP relaxation of OPF
As a first step, we present a simple implementation of the classical SDP formulation of the Optimnal Power Flow problem using CVXPY.

### Setup
We start by importing pandapower for the test cases;
CVXPY, an open source Python-embedded modeling language for convex optimization problems;
MOSEK, a large scale optimization software. 

In [3]:
import pandapower.networks as nw
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
import mosek

### 1st-Example
Let's first construct a small, 5-bus grid ourselves to really understand each step.

#### Define Variables and admittance matrix

In [2]:
n = 5  # Number of buses
G = [0, 1]  # Generator buses are either on or off
N = range(n)  # All buses
Y = np.random.randn(n, n) + 1j * np.random.randn(n, n)  # Admittance matrix
c2 = np.array([0.1, 0.2])  # Quadratic cost coefficients
c1 = np.array([10, 20])  # Linear cost coefficients
c0 = np.array([100, 200])  # Constant cost coefficients
P_min = np.array([0, 0])  # Minimum active power
P_max = np.array([100, 150])  # Maximum active power
Q_min = np.array([-50, -50])  # Minimum reactive power
Q_max = np.array([50, 50])  # Maximum reactive power
V_min = 0.95  # Minimum voltage magnitude
V_max = 1.05  # Maximum voltage magnitude
S_max = 200  # Maximum apparent power flow

#### Define Optimization Variables
Let's define the power flow constraint variables (P, Q) and the PSD matrix (X).

In [3]:
# Define optimization variables
X = cp.Variable((n, n), hermitian=True)  # X = VV^+
P = cp.Variable(len(G))  # Active power generation
Q = cp.Variable(len(G))  # Reactive power generation

#### Define Objective function

In [4]:
# Define objective
objective = cp.Minimize(cp.sum(c2 @ cp.square(P) + c1 @ P + c0))

#### Constraints

In [5]:
constraints = [X >> 0]  # X is positive semidefinite
for i in G:
    constraints += [P_min[i] <= P[i], P[i] <= P_max[i]]
    constraints += [Q_min[i] <= Q[i], Q[i] <= Q_max[i]]
for i in N:
    constraints += [V_min**2 <= cp.real(X[i, i]), cp.real(X[i, i]) <= V_max**2]
for i in N:
    for j in N:
        if i != j:
            S_ij = Y[i, j].conj() * X[i, j]
            constraints += [cp.abs(S_ij) <= S_max]

#### Solve the problem using cvxpy's SDP solvers. (probably IPM)

In [7]:
problem = cp.Problem(objective, constraints)
# Solve using a free solver (e.g., ECOS)
problem.solve()

# Print results
print("Optimal value:", problem.value)
print("X matrix:", X.value)
print("P:", P.value)
print("Q:", Q.value)

Optimal value: 300.0000000001779
X matrix: [[1.00235207+0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 1.00235207+0.j 0.        +0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 1.00235207+0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 1.00235207+0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  1.00235207+0.j]]
P: [2.82054517e-12 3.03767099e-12]
Q: [-50. -50.]


### Load Network Data
Now,let's make a function that loads the IEEE test cases from Pandapower.

In [4]:
# Select IEEE test case
def load_network(case):
    if case == 14:
        return nw.case14()
    elif case == 30:
        return nw.case30()
    elif case == 57:
        return nw.case57()
    elif case == 118:
        return nw.case118()
    else:
        raise ValueError("Unsupported test case")

In [41]:
# Load the chosen IEEE test case
case_number = 30 # Change to 30, 57, or 118 for larger cases
net = load_network(case_number)
pp.runpp(net)  # Run power flow to initialize values

When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accessed after power flow using net._ppc
We will get the bus admittance matrix using exactly that and extract all the parameters we need.

In [42]:
#Example of what the bus-branch model look like
# net._ppc 
net.load["q_mvar"].values

array([12.7,  1.2,  1.6, 10.9, 30. ,  2. ,  7.5,  1.6,  2.5,  1.8,  5.8,
        0.9,  3.4,  0.7, 11.2,  1.6,  6.7,  2.3,  0.9,  1.9])

In [43]:
# Now, we extract network parameters
Ybus = net._ppc["internal"]["Ybus"].todense() # Bus admittance matrix (we need todense since the matrices are stored as sparcse matrices, but dense are easier to work with)
n = len(net.bus)  # Number of buses
branch = net.line  # Get each transmission line's specification for line constraints

# Extract generator information for generation constraints
gen_buses = net.gen["bus"].values #.values convert the dataframe to numpy arrays
Pg_min = net.gen["min_p_mw"].values #minimal real power generation limit
Pg_max = net.gen["max_p_mw"].values #maximum real power generation limit
Q_min = net.gen["min_q_mvar"].values #minimal reactive power generation limit
Q_max = net.gen["max_q_mvar"].values #maximum real power generation limit

# Extract non-generator buses for generation constraints
Pd = np.zeros(n)  # Default to zero
Qd = np.zeros(n)
Pd[net.load["bus"].values] = net.load["p_mw"].values #load real power demand
Qd[net.load["bus"].values] = net.load["q_mvar"].values #load reactive power demand

# Voltage limits
V_min = net.bus["min_vm_pu"].values ** 2  # Squared for SDP
V_max = net.bus["max_vm_pu"].values ** 2

### Define Variable
Define the SDP Matrix X in CVXPY. We want X to be PSD. If X is rank-1, then our solution is the global optimal. In most power system cases, X is rank-1.
Represents voltage relations in SDP relaxation.

In [53]:
# Define an arbitrary n x n Hermitian X (Directly enforcing X= V V^+ will require X to be rank-1, which is too strict. We use the power balance, PSD and voltage constraints to guide this arbitrary X to X = VV^+)
X = cp.Variable((n, n), hermitian=True)

# Objective function: Minimize Pg, the matrix representing the real power generation at each generator 
Pg = cp.real(cp.trace(X @ Ybus))  # P_g = XY. The trace sums up all the power generation and estimate the cost. We also ensure Pg is real-valued


# Constraints
constraints = []

# Power balance equations (real and reactive)
for i in range(n):
    P_i = cp.real(Ybus[i, :] @ X @ Ybus[:,i])  # Extract real power injection at bus i
    Q_i = cp.imag(Ybus[i, :] @ X @ Ybus[:,i])  # Extract reactive power injection at bus i

    # If bus is a generator bus, enforce generation limits
    if i in gen_buses:
        idx = np.where(gen_buses == i)[0][0]  # Get generator index
        constraints.append(Pg_min[idx] <= P_i)
        constraints.append(P_i <= Pg_max[idx])
        constraints.append(Q_min[idx] <= Q_i)
        constraints.append(Q_i <= Q_max[idx])
    # else:
    #     # For non-generator buses, enforce power balance constraints
    #     constraints.append(P_i == -Pd[i])
    #     constraints.append(Q_i == -Qd[i])


# Voltage magnitude constraints
for i in range(n):
    constraints.append(V_min[i] <= cp.real(X[i, i]))
    constraints.append(cp.real(X[i, i]) <= V_max[i])

# Positive semidefinite constraint
constraints.append(X >> 0)

### Solve using cvxpy's SDP solver

In [54]:
# Solve the SDP
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

# Print results
print("Optimal cost:", prob.value)
print("Optimal voltage matrix X:\n", X.value)

Optimal cost: 0.0
Optimal voltage matrix X:
 [[ 1.07061827e+00+0.00000000e+00j  9.72943800e-01-1.12190572e-14j
  -2.27521143e-15+1.00112875e-15j  8.94205025e-01+5.57747077e-14j
   8.10468027e-01-1.75270147e-13j  8.95966045e-01+4.51070932e-14j
  -1.86786722e-15-1.67080662e-15j -1.30687306e-15-1.99760043e-15j
   1.17338926e-15+2.20835667e-15j -2.67793045e-15-2.85232761e-15j
  -1.54334342e-15+7.64247477e-17j -1.24124859e-15-2.98434558e-15j
  -1.24125185e-15-2.98432334e-15j -1.21063279e-15+8.84314707e-16j
  -2.95954729e-15-1.74086533e-15j  1.36647051e-15+1.68028165e-15j
   4.96557460e-16+6.15100987e-16j  3.36196887e-16+7.42173707e-16j
  -7.13301175e-16+2.74777104e-15j -1.16374360e-16+1.54441728e-15j
  -3.50859977e-15-4.25812698e-15j -3.76684538e-15-4.23140383e-15j
  -4.68367122e-15-3.19808361e-15j -6.94160112e-15-5.11427806e-15j
  -1.43822084e-15-1.57789012e-15j  6.49812478e-16+9.20329168e-16j
  -2.53087939e-15-1.65397998e-15j -3.93480303e-15-1.93270039e-15j
  -1.49995105e-15-1.01888713e-1

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");
